In [ ]:
!pip install transformers
!pip install datasets
!pip install sacrebleu

     |████████████████████████████████| 1.4MB 8.1MB/s 
     |████████████████████████████████| 890kB 25.7MB/s 
     |████████████████████████████████| 2.9MB 39.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=0f63011965e6cfadf2241c2d1148734c16b21e67b64d0ff87d181cac3f41d8c4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 163kB 8.8MB/s 
     |████████████████████████████████| 17.7MB 215kB/s 
     |████████████████████████████████| 245kB 53.6MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 71kB 5.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Training

In [ ]:
!python finetune_gp2v2.py \
    --model_name_or_path gpt2 \
    --train_file train_cate.txt \
    --validation_file test_cate.txt \
    --do_train \
    --do_eval \
    --output_dir gpt2_cate \
    --eval_steps 100 \
    --save_steps 5000 \
    --warmup_steps 500 \
    --per_device_train_batch_size=2 \
    --per_device_eval_batch_size=2 \
    --overwrite_output_dir \
    --num_train_epochs 1000

2020-12-16 05:44:21.705929: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/16/2020 05:44:26 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/16/2020 05:44:26 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/drive/MyDrive/yelp/gpt2_yelp', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1000.0, max_steps=-1, warmup_steps=500, logging_dir='runs/Dec16_05-44-26_e79243ec0e00', logging_first_step

# Computing BLEU Score

In [ ]:
from sacrebleu import corpus_bleu

def calculate_bleu(output_lns, refs_lns, **kwargs) -> dict:
    """Uses sacrebleu's corpus_bleu implementation."""
    return {"bleu": round(corpus_bleu(output_lns, [refs_lns], **kwargs).score, 4)}

In [ ]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_cate/checkpoint-10000")
GPT2 = TFGPT2LMHeadModel.from_pretrained("./gpt2_cate/checkpoint-10000", pad_token_id=tokenizer.eos_token_id, from_pt=True)
MAX_LEN = 70
with open('./test_cate_only.txt', 'r') as f:
    testdatas = f.readlines()
with open('./test_reviews.txt', 'r') as f:
    labels = f.readlines()

pred_strs = []
label_strs = []
for i, testdata in enumerate(testdatas):
    input_sequence = []
    input_sequence.append(testdata[:-1] + '<SEP>')
    # print(input_sequence)
    label_strs.append(labels[i][:-1])
    input_ids = tokenizer(input_sequence, add_special_tokens=False, return_tensors='tf').input_ids
    # generate text until the output length (which includes the context length) reaches 50
    greedy_output = GPT2.generate(input_ids, max_length=MAX_LEN)

    pred_strs.append(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

# print(pred_strs)
# print(label_strs)
bleu = calculate_bleu(pred_strs, label_strs)
print(bleu['bleu'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.4.attn.bias', 'transformer.h.6.attn.bias', 'transformer.h.9.attn.bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.5.attn.bias', 'transformer.h.1.attn.bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight', 'transformer.h.6.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.3.attn.bias', 'transformer.h.7.attn.bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.0.attn.bias', 'transformer.h.8.attn.bias', 'transformer.h.10.attn.bias', 'transformer.h.8.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on an

['food positive bread positive <SEP>']
['service positive delivery times positive <SEP>']
['food positive Food positive <SEP>']
['food positive coffee positive <SEP>']
['food conflict ambience positive sushi conflict place positive <SEP>']
['e lobster sandwich is good and the spaghetti with Scallops and Shrimp is great.  food positive food positive <SEP>', 'e lobster sandwich is good and the spaghetti with Scallops and Shrimp is great.  food positive food positive <SEP>', 'e lobster sandwich is good and the spaghetti with Scallops and Shrimp is great.  food positive food positive <SEP>', 'e lobster sandwich is good and the spaghetti with Scallops and Shrimp is great.  food positive food positive <SEP>', 'e lobster sandwich is good and the spaghetti with Scallops and Shrimp is great.  food positive food positive <SEP>']
['The bread is top notch as well.', 'I have to say they have one of the fastest delivery times in the city.', 'Food is always fresh and hot- ready to eat!', 'Did I menti

# Generating new reviews

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_cate/checkpoint-10000")
GPT2 = TFGPT2LMHeadModel.from_pretrained("./gpt2_cate/checkpoint-10000",
                                          pad_token_id=tokenizer.eos_token_id, from_pt=True)
MAX_LEN = 70
with open('./test_cate_only.txt', 'r') as f:
    testdatas = f.readlines()
with open('./test_reviews.txt', 'r') as f:
    labels = f.readlines()

pred_strs = []
label_strs = []
for testdata in testdatas:
    input_sequence = []
    input_sequence.append(testdata[:-1] + '<SEP>')
    input_ids = tokenizer.encode(input_sequence, return_tensors='tf')

    # generate text until the output length (which includes the context length) reaches 50
    greedy_output = GPT2.generate(input_ids, max_length=MAX_LEN)

    pred_strs.append(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

with open('./generated_review_cate.txt', 'w') as f:
    for i in range(len(pred_strs)):
        f.write(testdatas[i])
        f.write(label_strs[i])
        f.write(pred_strs[i] + '\n')